In [1]:
huggingface_token = ''
import os
os.environ["HF_TOKEN"] = str(huggingface_token)

In [7]:
class Args:
  def __init__(self):
    self.context_length = 512
    self.model = "meta-llama/Llama-2-7b-chat-hf"
    self.reinforced_model = "./models/reinforced"
    self.dict_file = "./data/dicts_new.npy"
    self.input_file = "/scratch/mk8475/one_book/raw_data.txt"
    self.output_file =  "mk8475/strac/data/generic_predictions.hf"
    self.bootstrap_coef = 5
    self.device = "cuda:0"

In [9]:
from transformers import AutoTokenizer
from datasets import Dataset

args = Args()  # Make sure Args is properly defined elsewhere
custom_cache_dir = "cache" 
tokenizer = AutoTokenizer.from_pretrained(args.model, cache_dir=custom_cache_dir)
CHUNK_SIZE = 10000
IGNORE_TOKEN_ID = -100  # Assuming you want to use the pad token as the ignore token

def read_file_into_chunks_and_prepare_labels(filename, context_length):
    # Initialize lists to store input_ids and labels for each chunk
    input_ids_list = []
    labels_list = []

    with open(filename, 'r', encoding='utf-8') as file:
        while True:
            chunk = file.read(CHUNK_SIZE)
            if not chunk:
                break

            # Tokenize the chunk
            chunk_tokens = tokenizer.encode(chunk, add_special_tokens=True)

            # Create overlapping blocks of tokens for input_ids and labels
            for i in range(0, len(chunk_tokens) - context_length + 1, context_length):
                input_ids = chunk_tokens[i:i+context_length]
                # Prepare labels by shifting them to the right
                labels = [IGNORE_TOKEN_ID] + chunk_tokens[i:i+context_length-1]

                input_ids_list.append(input_ids)
                labels_list.append(labels)

    return input_ids_list, labels_list

input_ids, labels = read_file_into_chunks_and_prepare_labels(args.input_file, args.context_length)

# Preparing the dataset for saving
dataset = Dataset.from_dict({
    'input_ids': input_ids,
    'labels': labels,
})

# Save the dataset to disk
dataset.save_to_disk("/scratch/mk8475/one_book/raw_text.hf")


Saving the dataset (0/1 shards):   0%|          | 0/229 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset

dataset = Dataset.from_dict(chunks)

# Main processing of dataset
dataset.save_to_disk(f"/LLM/raw_text.hf")


In [10]:
dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 229
})